## Scraping Research Scientist jobs from Seek.com.au using Beautiful Soup

In [1]:
# Importing libraries
import re
import pandas as pd
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
from bs4 import BeautifulSoup
import json
from time import sleep

In [2]:
# Choosing two salary ranges for each job, which are the equivalent of Low and High
salary_ranges = ['50000-80000', '80000-120000', '120000-150000']

# Dictionary to store all the web links
job_urls = {}

for salary_range in salary_ranges:
    job_urls[salary_range]=[]
    for page in range(1,40):
        # iterate through the first 40 pages of jobs
        r = requests.get("https://www.seek.com.au/research-scientist-jobs/in-All-Sydney-NSW?page={0}&salaryrange={1}&salarytype=annual".format(str(page),salary_range))

        # turn into a BeautifulSoup object
        soup = BeautifulSoup(r.text, 'lxml')

        # find urls on page
        url = soup.find_all(attrs={'data-automation':"jobTitle"})
        /
        if url != []: 
            job_urls[salary_range].extend(url)

In [3]:
df_1 = pd.DataFrame()
df_1['Link'] = job_urls['50000-80000']
df_1['Salary Range'] = '50000-80000'

In [4]:
df_1.head()

,Link,Salary Range
0,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
1,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
2,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
3,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000
4,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000


In [5]:
df_2 = pd.DataFrame()
df_2['Link'] = job_urls['80000-120000']
df_2['Salary Range'] = '80000-120000'

In [6]:
df_3 = pd.DataFrame()
df_3['Link'] = job_urls['120000-150000']
df_3['Salary Range'] = '120000-150000'

In [7]:
url_pd = df_1.append(df_2).append(df_3)

In [8]:
url_pd.tail()

,Link,Salary Range
125,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
126,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
127,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
128,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000
129,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",120000-150000


In [9]:
url_pd['Link_1'] = ['https://www.seek.com.au' + re.search('(\/job\/[0-9]{8})',i.attrs['href']).group(0) for i in url_pd['Link']]

In [10]:
url_pd.head()

,Link,Salary Range,Link_1
0,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40274554
1,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40219468
2,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40157423
3,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40199564
4,"<a class=""_2iNL7wI"" data-automation=""jobTitle""...",50000-80000,https://www.seek.com.au/job/40219469


In [11]:
url_pd['Title'] = [i.text for i in url_pd['Link']]

In [12]:
url_pd = url_pd.drop('Link',axis='columns')

In [13]:
url_pd.head()

,Salary Range,Link_1,Title
0,50000-80000,https://www.seek.com.au/job/40274554,"Research Scientist, Building Materials"
1,50000-80000,https://www.seek.com.au/job/40219468,Research Scientist
2,50000-80000,https://www.seek.com.au/job/40157423,Research Scientist | Building Materials
3,50000-80000,https://www.seek.com.au/job/40199564,Research Scientist (Machine Learning)
4,50000-80000,https://www.seek.com.au/job/40219469,Research Scientist


In [14]:
columns = ['Salary Range', 'Link', "Title"]
url_pd.columns=columns

In [15]:
url_pd.head()

,Salary Range,Link,Title
0,50000-80000,https://www.seek.com.au/job/40274554,"Research Scientist, Building Materials"
1,50000-80000,https://www.seek.com.au/job/40219468,Research Scientist
2,50000-80000,https://www.seek.com.au/job/40157423,Research Scientist | Building Materials
3,50000-80000,https://www.seek.com.au/job/40199564,Research Scientist (Machine Learning)
4,50000-80000,https://www.seek.com.au/job/40219469,Research Scientist


In [16]:
#for stemming
import re

def replace(string, substitutions):

    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile('|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

In [17]:
job_listing_date=[]
job_expiry_date=[]
job_title = []
job_teaser=[]
job_advertiser=[]
job_area=[]
job_worktype=[]
job_classification=[]
job_salary=[]
job_salary_type=[]
job_description_details=[]
job_description_strong = []
job_location =[]

for index, job in enumerate(url_pd['Link']):
    # iterate through all the urls
    try:
        r = requests.get(job)
    except:
        time.sleep(5)
    
    # turn into a BeautifulSoup object
    soup = BeautifulSoup(r.text, 'lxml')
    
    # find job descriptions
    jd_data = soup.find_all(attrs={'data-automation':'server-state'})[0]
    jd_raw= jd_data.text[3:].split('\n  ')[1][25:][:-1]
    jd_dict = json.loads(jd_raw)
    jd_dashboard = jd_dict['jobdetails']['result']
    
    if jd_dashboard['listingDate']:
        job_listing_date.append(jd_dashboard['listingDate'])
    else:
        job_listing_date.append('N/A')
        
    if jd_dashboard['expiryDate']:
        job_expiry_date.append(jd_dashboard['expiryDate'])
    else:
        job_expiry_date.append('N/A')
        
    if jd_dashboard['title']:
        job_title.append(jd_dashboard['title'])
    else:
        job_title.append('N/A')
        
    if jd_dashboard['teaser']:
        job_teaser.append(jd_dashboard['teaser'])
    else:
        job_teaser.append('N/A')
        
    if jd_dashboard['advertiser']['description']:
        job_advertiser.append(jd_dashboard['advertiser']['description'])
    else:
        job_advertiser.append('N/A')
        
    if jd_dashboard['locationHierarchy']['area']:
        job_area.append(jd_dashboard['locationHierarchy']['area'])
    else:
        job_area.append('N/A')
        
    if jd_dashboard['workType']:
        job_worktype.append(jd_dashboard['workType'])
    else:
        job_worktype.append('N/A')
        
    if jd_dashboard['classification']['description']:
        job_classification.append(jd_dashboard['classification']['description'])
    else:
        job_classification.append('N/A')
        
    if jd_dashboard['salary']:
        job_salary.append(jd_dashboard['salary'])
    else:
        job_salary.append('N/A')
        
    if jd_dashboard['salaryType']:
        job_salary_type.append(jd_dashboard['salaryType'])
    else:
        job_salary_type.append('N/A')
        
    if jd_dashboard['locationHierarchy']['city']:
        job_location.append(jd_dashboard['locationHierarchy']['city'])
    else:
        job_location.append('N/A')
        
    
    jd_detail = soup.find(attrs={'data-automation':'mobileTemplate'})
    job_description_details.append(jd_detail)
    

In [18]:
url_pd['Listing Date'] = job_listing_date
url_pd['Expiry Date'] =job_expiry_date
url_pd['Job Title']=job_title
url_pd['Job Teaser']=job_teaser
url_pd['Advertiser'] = job_advertiser
url_pd['Area'] = job_area
url_pd['Work Type'] = job_worktype
url_pd['Classification'] = job_classification
url_pd['Salary'] = job_salary
url_pd['Salary Type'] = job_salary_type
url_pd['JD'] = job_description_details
url_pd['Location'] =job_location

In [19]:
# Check the type for 'JD' column and start cleaning
type(job_description_details)

list

In [20]:
job_description_clean=[]
job_description_strong = []
for i in range(0,len(url_pd['JD'])):
    try:
        strong_word = job_description_details[i].find_all('strong')
        strong_word_list = [u.get_text(strip=True) for u in strong_word if strong_word != False]
        job_description_strong.append(strong_word_list)

        string = job_description_details[i].get_text()
        repla = {u'\xa0':'  ',u'\xe2\x80\x9d':'  ', u'\n':'  '}
        job_description_clean.append(replace(string,repla))
    except:
        job_description_strong.append('N/A')
        job_description_clean.append('N/A')

In [21]:
url_pd['Job Description'] = job_description_clean
url_pd['Strong Words'] = job_description_strong 

In [22]:
research_scientist_jobs = url_pd.copy()

In [23]:
research_scientist_jobs.to_csv('research_scientist_jobs.csv')

In [24]:
research_scientist_jobs.tail(10)

,Salary Range,Link,Title,Listing Date,Expiry Date,Job Title,Job Teaser,Advertiser,Area,Work Type,Classification,Salary,Salary Type,JD,Location,Job Description,Strong Words
120,120000-150000,https://www.seek.com.au/job/40130429,"Technical Account Manager, Speciality Chemical...",2019-10-09T23:36:08.000Z,2019-11-09T12:59:59.000Z,"Technical Account Manager, Speciality Chemical...",Seeking Technical Account Manager with knowled...,Evolve Scientific Recruitment,North Shore & Northern Beaches,Full Time,Sales,"car allowance, fuel card, bonus",AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,About the company: This well-established Au...,"[About the company:, About the role:, strategi..."
121,120000-150000,https://www.seek.com.au/job/40130551,"Technical Account Manager, Speciality Chemical...",2019-10-09T23:46:06.000Z,2019-11-09T12:59:59.000Z,"Technical Account Manager, Speciality Chemical...",Seeking Technical Account Manager with knowled...,Evolve Scientific Recruitment,North Shore & Northern Beaches,Full Time,Sales,"+ car allowance, fuel card, bonus",AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,About the company:This well-established Austra...,"[About the company:, About the role:, strategi..."
122,120000-150000,https://www.seek.com.au/job/40067220,Environmental Coordinator,2019-10-01T04:49:25.000Z,2019-10-31T04:49:24.000Z,Environmental Coordinator,A dynamic and fast growing second generation C...,Private Advertiser,"CBD, Inner West & Eastern Suburbs",Full Time,Construction,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,Burton Contractors is a leading Civil Contract...,"[Expected Deliverables:, employment@burtoncont..."
123,120000-150000,https://www.seek.com.au/job/40084078,Site Lean Lead,2019-10-03T01:28:45.000Z,2019-11-02T12:59:59.000Z,Site Lean Lead,Drive Operational Excellence as a member of th...,AstraZeneca,Ryde & Macquarie Park,Full Time,"Manufacturing, Transport & Logistics",N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,At AstraZeneca we believe that innovation need...,"[Role, You, Apply Today !!!]"
124,120000-150000,https://www.seek.com.au/job/40072750,"DevOps Architect, Windows, Automation, Design,...",2019-10-01T23:52:32.000Z,2019-10-31T23:52:32.000Z,"DevOps Architect, Windows, Automation, Design,...","Architect, Design & Automate Windows Infrastru...",Westbury Partners,"CBD, Inner West & Eastern Suburbs",Full Time,Information & Communication Technology,$150K - $200K+ + Bonus,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,CompanyOur Client is a leading proprietary tra...,"[Company, Team, Role, Requirements:]"
125,120000-150000,https://www.seek.com.au/job/40073170,Principal Analyst,2019-10-02T00:43:26.000Z,2019-11-01T12:59:59.000Z,Principal Analyst,The Principal Analyst will be responsible for ...,Independent Pricing and Regulatory Tribunal,N/A,Full Time,"Mining, Resources & Energy",$127424 - $135288 p.a.,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,Principal Analyst - IPART Officer FFulltime on...,"[Principal Analyst - IPART Officer F, Fulltime..."
126,120000-150000,https://www.seek.com.au/job/40076093,People and Change Senior Consultant,2019-10-02T03:48:45.000Z,2019-11-01T12:59:59.000Z,People and Change Senior Consultant,An exciting career opportunity for experienced...,Aurecon,N/A,Full Time,Consulting & Strategy,N/A,AnnualPackage,"<div class=""_2e4Pi2B"" data-automation=""mobileT...",Sydney,Just imagine your future with us… At Aurecon ...,"[Just imagine your future with us…, What will ..."
127,120000-150000,https://www.seek.com.au/job/40065052,Technology Project Manager - Software Medical ...,2019-10-01T02:22:58.000Z,2019-10-31T02:22:57.000Z,Technology Project Manager - Software Medical ...,Join our innovative Software Engineering team ...,Corin Australia,North Shore & Northern Beaches,Full Time,Information & Communication Technology,N/A